In [ ]:
%pip install transformers adapters torch datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 45.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.4
    Uninstalling transformers-4.55.4:
      Successfully uninstalled transformers-4.55.4


In [ ]:
from transformers import AutoTokenizer
from adapters import AutoAdapterModel

model_name = "GroNLP/hateBERT"  # Replace with the correct HateBERT model path
model = AutoAdapterModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertAdapterModel were not initialized from the model checkpoint at GroNLP/hateBERT and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/151 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# model.add_adapter("hate_task")
# model.add_classification_head("hate_task", num_labels=3)

model.add_adapter("mh_task")
model.add_classification_head("mh_task", num_labels=3)


In [ ]:
from datasets import load_dataset

# hate_ds = load_dataset("csv", data_files="./hd_final.csv")
mh_ds = load_dataset("csv", data_files="./mhd_final.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from datasets import DatasetDict

# Split the hate dataset
# hate_train_test = hate_ds['train'].train_test_split(test_size=0.2, seed=42)
# hate_test_valid = hate_train_test['test'].train_test_split(test_size=0.5, seed=42)
# hate_datasets = DatasetDict({
#     'train': hate_train_test['train'],
#     'validation': hate_test_valid['train'],
#     'test': hate_test_valid['test']
# })

# Split the mental health dataset
mh_train_test = mh_ds['train'].train_test_split(test_size=0.2, seed=42)
mh_test_valid = mh_train_test['test'].train_test_split(test_size=0.5, seed=42)
mh_datasets = DatasetDict({
    'train': mh_train_test['train'],
    'validation': mh_test_valid['train'],
    'test': mh_test_valid['test']
})

In [ ]:
# Make sure label columns exist and are integers
def prepare_labels(example, label_column):
    example["labels"] = int(example[label_column])
    return example

# Replace 'label' with whatever your CSV column is
# hate_datasets = hate_datasets.map(lambda x: prepare_labels(x, label_column="labels"))
mh_datasets = mh_datasets.map(lambda x: prepare_labels(x, label_column="labels"))

Map:   0%|          | 0/42144 [00:00<?, ? examples/s]

Map:   0%|          | 0/5268 [00:00<?, ? examples/s]

Map:   0%|          | 0/5269 [00:00<?, ? examples/s]

In [ ]:
def tokenize_fn(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Tokenize hate_datasets
# hate_datasets = hate_datasets.map(tokenize_fn, batched=True)
# Tokenize mh_datasets
mh_datasets = mh_datasets.map(tokenize_fn, batched=True)


Map:   0%|          | 0/42144 [00:00<?, ? examples/s]

Map:   0%|          | 0/5268 [00:00<?, ? examples/s]

Map:   0%|          | 0/5269 [00:00<?, ? examples/s]

In [ ]:
print(set(hate_datasets["train"]["labels"]))
print(set(mh_datasets["train"]["labels"]))

NameError: name 'hate_datasets' is not defined

In [ ]:
from transformers import Trainer, TrainingArguments
import torch

# Check for available device
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    eval_strategy="epoch",
    remove_unused_columns=False,  # Important for adapters
    learning_rate=2e-5,
    warmup_steps=500,
    logging_steps=100,
    # GPU acceleration settings
    dataloader_pin_memory=False,  # Set to False for MPS to avoid issues
    fp16=False,  # Set to False for MPS (fp16 not supported on MPS)
    report_to=[]
)

# Move model to GPU
model = model.to(device)

# # --- Hate dataset ---
# model.set_active_adapters("hate_task")
# model.train_adapter("hate_task")  # Enable adapter training

# trainer_hate = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=hate_datasets["train"],
#     eval_dataset=hate_datasets["test"],
# )
# trainer_hate.train()

# --- Mental health dataset ---
model.set_active_adapters("mh_task")
model.train_adapter("mh_task")  # Enable adapter training

trainer_mh = Trainer(
    model=model,
    args=training_args,
    train_dataset=mh_datasets["train"],
    eval_dataset=mh_datasets["test"],
)
trainer_mh.train()

Using device: cuda


Epoch,Training Loss,Validation Loss
1,0.267000,No log
2,0.283100,No log
3,0.252600,No log


TrainOutput(global_step=15804, training_loss=0.270081556136021, metrics={'train_runtime': 9703.3875, 'train_samples_per_second': 13.03, 'train_steps_per_second': 1.629, 'total_flos': 3.4084611609624576e+16, 'train_loss': 0.270081556136021, 'epoch': 3.0})

In [ ]:
# Save just the adapter for hate_task
# model.save_adapter("./hate_adapter", "hate_task")

# Save just the adapter for mh_task
model.save_adapter("./mh_adapter", "mh_task")